# Implémentation d'un extracteur de caractéristiques à l'aide d'un réseau pré-entraîné

L'un des moyens les plus simples de profiter de la puissance de l'apprentissage par transfert est d'utiliser des modèles pré-entraînés comme extracteurs de caractéristiques. De cette façon, nous pouvons combiner à la fois l'apprentissage en profondeur et l'apprentissage automatique, ce que nous ne pouvons normalement pas faire, car les algorithmes d'apprentissage automatique traditionnels ne fonctionnent pas avec des images brutes. Dans cette recette, nous allons implémenter une classe FeatureExtractor réutilisable pour produire un ensemble de données de vecteurs à partir d'un ensemble d'images d'entrée, puis l'enregistrer au format HDF5 incroyablement rapide

Nous utiliserons l'ensemble de données Stanford Cars, que vous pouvez télécharger ici : https://www.kaggle.com/jessicali9530/stanford-cars-dataset. Décompressez les données à l'emplacement de votre choix.

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rachik","key":"2ed63b5dedc5fe382387a65e3c50836c"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [4]:
!kaggle datasets download -d jessicali9530/stanford-cars-dataset

 99% 1.82G/1.82G [00:43<00:00, 28.1MB/s]
100% 1.82G/1.82G [00:43<00:00, 44.8MB/s]


In [5]:
!unzip stanford-cars-dataset.zip

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: cars_train/cars_train/03145.jpg  
  inflating: cars_train/cars_train/03146.jpg  
  inflating: cars_train/cars_train/03147.jpg  
  inflating: cars_train/cars_train/03148.jpg  
  inflating: cars_train/cars_train/03149.jpg  
  inflating: cars_train/cars_train/03150.jpg  
  inflating: cars_train/cars_train/03151.jpg  
  inflating: cars_train/cars_train/03152.jpg  
  inflating: cars_train/cars_train/03153.jpg  
  inflating: cars_train/cars_train/03154.jpg  
  inflating: cars_train/cars_train/03155.jpg  
  inflating: cars_train/cars_train/03156.jpg  
  inflating: cars_train/cars_train/03157.jpg  
  inflating: cars_train/cars_train/03158.jpg  
  inflating: cars_train/cars_train/03159.jpg  
  inflating: cars_train/cars_train/03160.jpg  
  inflating: cars_train/cars_train/03161.jpg  
  inflating: cars_train/cars_train/03162.jpg  
  inflating: cars_train/cars_train/03163.jpg  
  inflating: cars_train/cars_

Nous stockerons les caractéristiques extraites au format HDF5, un protocole hiérarchique binaire conçu pour stocker de très grands ensembles de données numériques sur disque, tout en conservant la facilité d'accès et de calcul au niveau des lignes. Vous pouvez en savoir plus sur HDF5 ici : https://portal.hdfgroup.org/display/HDF5/HDF5.

Suivez ces étapes pour terminer cette recette :

**1.** Importez tous les packages nécessaires :

In [6]:
import glob
import os
import pathlib

import h5py
import numpy as np
import sklearn.utils as skutils
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import *
from tqdm import tqdm

**2.** Dénissez la classe FeatureExtractor et son constructeur :

In [7]:
class FeatureExtractor(object):
    def __init__(self,
                 model,
                 input_size,
                 label_encoder,
                 num_instances,
                 feature_size,
                 output_path,
                 features_key='features',
                 buffer_size=1000):
      
        #Nous devons nous assurer que le chemin de sortie peut être écrit

        if os.path.exists(output_path):
            error_msg = (f'{output_path} already exists. '
                         f'Please delete it and try again.')
            raise FileExistsError(error_msg)
        # Maintenant, stockons le paramètre d'entrée en tant objet
        self.model = model
        self.input_size = input_size
        self.le = label_encoder
        self.feature_size = feature_size
        
        # self.buffer contiendra un tampon d'instances et d'étiquettes, 
        #tandis que self.current_index pointera vers le prochain emplacement
        # libre dans les ensembles de données de la base de données interne HDF5. Nous allons créer ceci maintenant :

        self.db = h5py.File(output_path, 'w')
        self.features = self.db.create_dataset(features_key,
                                               (num_instances,
                                                feature_size),
                                               dtype='float')
        
        self.labels = self.db.create_dataset('labels',
                                             (num_instances,),
                                             dtype='int')

        self.buffer_size = buffer_size
        self.buffer = {'features': [], 'labels': []}
        self.current_index = 0

    # Définir une méthode qui extraira les caractéristiques et les étiquettes d'une liste de chemins d'images et les stockera dans la base de données HDF5
    def extract_features(self,
                         image_paths,
                         labels,
                         batch_size=64,
                         shuffle=True):
        if shuffle:
            image_paths, labels = skutils.shuffle(image_paths,
                                                  labels)
            
        encoded_labels = self.le.fit_transform(labels)
        self._store_class_labels(self.le.classes_)

        # Après avoir mélangé les chemins d'images et leurs étiquettes,
        # ainsi que l'encodage et le stockage de ces derniers, nous itérerons sur des lots d'images, en les passant à travers le réseau pré-entraîné. 
        #Une fois cela fait, nous enregistrerons les fonctionnalités résultantes dans la base de données HDF5 (les méthodes d'aide que nous avons utilisées ici seront définies sous peu)

        for i in tqdm(range(0, len(image_paths), batch_size)):

            batch_paths = image_paths[i: i + batch_size]
            batch_labels = encoded_labels[i:i + batch_size]
            batch_images = []

            for image_path in batch_paths:
                image = load_img(image_path,
                                 target_size=self.input_size)
                image = img_to_array(image)
                image = np.expand_dims(image, axis=0)
                image = imagenet_utils.preprocess_input(image)

                batch_images.append(image)

            batch_images = np.vstack(batch_images)
            feats = self.model.predict(batch_images,
                                       batch_size=batch_size)

            new_shape = (feats.shape[0], self.feature_size)
            feats = feats.reshape(new_shape)
            self._add(feats, batch_labels)

        self._close()
    # Définir une méthode privée qui ajoutera des entités et des étiquettes aux jeux de données correspondants
    def _add(self, rows, labels):
        self.buffer['features'].extend(rows)
        self.buffer['labels'].extend(labels)

        if len(self.buffer['features']) >= self.buffer_size:
            self._flush()
    # Définir une méthode privée qui videra le buffer sur le disque
    def _flush(self):
        next_index = (self.current_index +
                      len(self.buffer['features']))
        buffer_slice = slice(self.current_index, next_index)
        self.features[buffer_slice] = self.buffer['features']
        self.labels[buffer_slice] = self.buffer['labels']
        self.current_index = next_index
        self.buffer = {'features': [], 'labels': []}
    # Définir une méthode privée qui stockera les étiquettes de classe dans la base de données HDF5
    def _store_class_labels(self, class_labels):
        data_type = h5py.special_dtype(vlen=np.dtype('int32'))
        label_ds = self.db.create_dataset('label_names',
                                          (len(class_labels),),
                                          dtype=data_type)
        label_ds[:] = class_labels
    # Définir une méthode privée qui fermera le jeu de données HDF5
    def _close(self):
        if len(self.buffer['features']) > 0:
            self._flush()

        self.db.close()

**3.** Charger les chemins vers les images dans le jeu de données

In [8]:
files_pattern = (pathlib.Path('/content/') /'cars_train/cars_train' / '*.jpg')
files_pattern = str(files_pattern)
input_paths = [*glob.glob(files_pattern)]

**4.** Créez le répertoire de sortie. Nous allons créer un ensemble de données d'images de voitures tournées afin qu'un classificateur potentiel puisse apprendre à rétablir correctement les photos dans leur orientation d'origine, en prédisant correctement l'angle de rotation.

In [9]:
output_path = pathlib.Path('/content/') / 'car_ims_rotated'

**5.**Créez une copie du jeu de données avec des rotations aléatoires effectuées sur les images :

In [10]:
if not os.path.exists(str(output_path)):
    os.mkdir(str(output_path))

labels = []
output_paths = []
for index in tqdm(range(len(input_paths))):
    image_path = input_paths[index]
    image = load_img(image_path)
    rotation_angle = np.random.choice([0, 90, 180, 270])

    rotated_image = image.rotate(rotation_angle)
    rotated_image_path = str(output_path / f'{index}.jpg')
    rotated_image.save(rotated_image_path, 'JPEG')

    output_paths.append(rotated_image_path)
    labels.append(rotation_angle)

    image.close()
    rotated_image.close()

100%|██████████| 8144/8144 [03:20<00:00, 40.53it/s]


**6.** Instanciez FeatureExtractor tout en utilisant un réseau VGG16 pré-entraîné pour extraire des caractéristiques des images de l'ensemble de données


In [11]:
features_path = str(output_path / 'features.hdf5')
model = VGG16(weights='imagenet', include_top=False)
fe = FeatureExtractor(model=model,
                      input_size=(224, 224, 3),
                      label_encoder=LabelEncoder(),
                      num_instances=len(input_paths),
                      feature_size=512 * 7 * 7,
                      output_path=features_path)

58900480/58889256 [==============================] - 1s 0us/step


**7.** Extrayez les caractéristiques et les étiquettes :

In [12]:
fe.extract_features(image_paths=output_paths,
                    labels=labels)

100%|██████████| 128/128 [03:08<00:00,  1.47s/it]


Après quelques minutes, il devrait y avoir un fichier nommé features.hdf5 dans //content/car_ims_rotated

Dans cette recette, nous avons implémenté un composant réutilisable afin d'utiliser des réseaux pré-entraînés sur ImageNet, tels que VGG16 et ResNet, comme extracteurs de caractéristiques. C'est un excellent moyen d'exploiter les connaissances codées dans ces modèles, car cela nous permet d'utiliser les vecteurs de haute qualité résultants pour former des modèles d'apprentissage automatique traditionnels tels que la régression logistique et les machines à vecteurs de support.

Étant donné que les ensembles de données d'images ont tendance à être trop volumineux pour être stockés en mémoire, nous avons eu recours au format HDF5 hautes performances et convivial, idéal pour stocker de grandes données numériques sur disque, tout en conservant la facilité d'accès typique de NumPy. . Cela signifie que nous pouvons interagir avec les ensembles de données HDF5 comme s'il s'agissait de tableaux NumPy normaux, ce qui les rend compatibles avec l'ensemble de l'écosystème SciPy.

Le résultat de FeatureExtractor est un fichier HDF5 hiérarchique (considérez-le comme un dossier dans un système de fichiers) contenant trois ensembles de données : features, qui contient les vecteurs de fonctionnalités, labels, qui stockent les labels codés, et label_names, qui contient le -étiquettes lisibles avant l'encodage.

Enfin, nous avons utilisé FeatureExtractor pour créer une représentation binaire d'un ensemble de données d'images de voitures pivotées de 0º, 90º, 180º ou 270º.